In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 64.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=ee1df336ea7f8b91ccb8c7e2873175bc9440b0cd20cc79e8979ddd68e4ecc2db
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [ ]:
import numpy as np
import nltk
import pymorphy2
from typing import List

from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

from sklearn.feature_extraction.text import TfidfVectorizer
from numpy.linalg import norm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Функция для предобработки запроса и описаний книг. На входе и на выходе -- список из строк.

In [ ]:
def preprocess_corpus(corpus: List[str]):

    russian_stopwords = stopwords.words("russian")
    tokenized_corpus = [[token for token in nltk.word_tokenize(str(document).lower()) if token not in russian_stopwords and token.isalnum()] for document in corpus]

    morph = pymorphy2.MorphAnalyzer()
    lemmatized_corpus = [[morph.parse(token)[0].normal_form for token in document] for document in tokenized_corpus]
    preprocessed_corpus = [" ".join(tokens) for tokens in lemmatized_corpus]

    return preprocessed_corpus

Допустим, у нас есть стартовый набор из описаний пяти книг, который еще не был предобработан.

In [ ]:
descriptions = ["Графический роман основан на сюжете книги «Неправильные»! Это канон! Автор — Скотт Коутон, именно он разработал игру ФНАФ и написал книги. Полностью повторяет американское издание. Эту книгу мы с вами ждали 3 года!", "Автор объединил пять сказочных сюжетов из разных стран: кроме знакомых нашему читателю историй о Золушке и Чумазом напарнике дьявола представлены две японские и одна китайская сказка. Мастерство и изобретательность, присущие Дайдзиро Морохоси, позволили ему воплотить свои замыслы столь живо и убедительно, что почти с первой страницы читатель невольно начинает сам участвовать в действии и вклиниваться в разговоры персонажей. Такова сила сказки, помноженной на талант.", "Действие комикса «Бэтмен: Белый Рыцарь» разворачивается в мире, где главный клоун-маньяк вселенной DC, терроризировавший Готэм на протяжении многих лет, неожиданно исцеляется от безумия. В центре повествования – ранее Джокер, а ныне Джек Напьер, который собирается во что бы то ни стало избавить любимый город от всевозможных пороков. Воссоединившись со своей подругой Харли Квинн, Напьер приводит в исполнение тщательно продуманный план по дискредитации Бэтмена и возведению себя на пьедестал всенародной любви. Но грехи прошлого не дадут Напьеру воплотить задуманное в жизнь. Все, чего он добился, начинается рушиться на глазах, как карточный домик. Ведь на место одного Джокера всегда придет другой, и одолеть его можно только проверенными методами. Популярный сценарист и художник Шон Мерфи в истории «Бэтмен: Белый Рыцарь» заглядывает в душу величайшего злодея во вселенной DC. В этой невероятной истории собрано восемь выпусков одноименной мини-серии, ставшей дебютом «DC Black Label» – импринта, в котором всемирно известные сценаристы и художники представят ключевых персонажей издательства такими, какими вы никогда их прежде не видели", "История становления Бэтмена - впервые на русском языке! Фрэнк Миллер и Дэвид Мацукелли рассказывают историю становления Тёмного рыцаря в комиксе “Бэтмен. Год первый”. Брюсу Уэйну суждено стать легендой, но пока он просто хочет сделать свой город лучше. Что заставило его надеть маску и в первый раз выйти на мрачные и жестокие улицы Готэма? И что делать честному полицейскому Джиму Гордону — стремиться изменить мир к лучшему или стать частью коррумпированной системы? “Бэтмен. Год первый” — стал одним из ключевых сюжетов для графических историй, после которого не только комиксы о Бэтмене, но и вся супергероика взяла курс на мрачность и серьезность. В российское издание входит полная версия графического романа “Бэтмен. Год первый”, а также многочисленные дополнительные материалы: страницы сценария Фрэнка Миллера, зарисовки и наброски Дэвида Мацукелли, предисловие от обоих авторов.", "Двор Сов… Бэтмен слышал о нем. Все слышали, каждый ребенок знает стишок: «Берегитесь: Двор Совиный неустанно следит…» Говорят, Совы – настоящие хозяева Готэма, хоть никто их и не видел. Говорят, они вершат свой суд над неугодными. Говорят, от них не скроешься. Много чего говорят… Темный Рыцарь не верит в эти слухи. Что бы там не болтали, считает он, Готэм – город Бэтмена. Однако вскоре ему предстоит понять, как глубоко он заблуждался. Мрачные легенды не лгут: Готэмом втайне правят могущественные хищники, и гнезда их повсюду. Звездный дуэт писателя СКОТТА СНАЙДЕРА и художника ГРЕГА КАПУЛЛО с гордостью представляет «Бэтмен: Двор Сов» – грандиозная история о загадках и ужасах Готэма, ставшая одной из самых заметных в комиксах о Темном Рыцаре."]

Предобрабатываем.

In [ ]:
preprocessed_descriptions = preprocess_corpus(descriptions)
preprocessed_descriptions

['графический роман основать сюжет книга неправильный это канон автор скотт коутон именно разработать игра фнаф написать книга полностью повторять американский издание книга вы ждать 3 год',
 'автор объединить пять сказочный сюжет разный страна кроме знакомый наш читатель история золушка чумазый напарник дьявол представить два японский один китайский сказка мастерство изобретательность присущий дайдзиро морохося позволить воплотить свой замысел столь живо убедительно первый страница читатель невольно начинать участвовать действие вклиниваться разговор персонаж таков сила сказка помножить талант',
 'действие комикс бэтмена белый рыцарь разворачиваться мир главный вселенная dc терроризировать готэма протяжение многий год неожиданно исцеляться безумие центр повествование ранее джокер ныне джек напьер который собираться стать избавить любимый город всевозможный порок воссоединиться свой подруга харли квинна напьер приводить исполнение тщательно продумать план дискредитация бэтмена возведен

Теперь preprocessed_descriptions -- хранилище предобработанных описаний книг.

В случае изменения описаний книг или добавления новых стоит предобрабатывать эти описания отдельно, а затем добавлять их на нужную позицию в списке preprocessed_descriptions. 

Теперь, допустим, нам пришел запрос "бэтмен готэм джокер". Тоже произведем его предобработку.

In [ ]:
query = 'бэтмен готэм джокер'
query = [query]
preprocessed_query = preprocess_corpus(query)
preprocessed_query

['бэтмена готэма джокер']

Функция поиска. На вход принимает предобработанные запрос и описания книг, а также список ID книг, соответственно списку описаний книг.

In [ ]:
def search_books(query: List[str], descriptions: List[str], descriptions_ids: List[int]):

    corpus = query + descriptions
    
    tfIdfVectorizer = TfidfVectorizer(use_idf = True)
    tfIdf = tfIdfVectorizer.fit_transform(corpus).toarray()

    query_vector = tfIdf[0]
    descriptions_vectors = tfIdf[1:]

    cosine_similarity = np.dot(descriptions_vectors, query_vector) / (norm(descriptions_vectors, axis=-1) * norm(query_vector))
    sorted_similarity = cosine_similarity.argsort()[::-1]

    result = np.array(descriptions_ids)[sorted_similarity]

    return list(result)

In [ ]:
# список ID книг
descriptions_ids = list(range(5))

In [ ]:
# ищем книги
searched_books = search_books(preprocessed_query, preprocessed_descriptions, descriptions_ids)
searched_books

[2, 4, 3, 1, 0]

In [ ]:
for i in searched_books:
    print(descriptions[i])
    print('---------------------------------------------------------------------------------------------------------------------')

Действие комикса «Бэтмен: Белый Рыцарь» разворачивается в мире, где главный клоун-маньяк вселенной DC, терроризировавший Готэм на протяжении многих лет, неожиданно исцеляется от безумия. В центре повествования – ранее Джокер, а ныне Джек Напьер, который собирается во что бы то ни стало избавить любимый город от всевозможных пороков. Воссоединившись со своей подругой Харли Квинн, Напьер приводит в исполнение тщательно продуманный план по дискредитации Бэтмена и возведению себя на пьедестал всенародной любви. Но грехи прошлого не дадут Напьеру воплотить задуманное в жизнь. Все, чего он добился, начинается рушиться на глазах, как карточный домик. Ведь на место одного Джокера всегда придет другой, и одолеть его можно только проверенными методами. Популярный сценарист и художник Шон Мерфи в истории «Бэтмен: Белый Рыцарь» заглядывает в душу величайшего злодея во вселенной DC. В этой невероятной истории собрано восемь выпусков одноименной мини-серии, ставшей дебютом «DC Black Label» – импринт